In [1]:
import networkx as nx
import hashlib

def hash_function(data):
    """Generate a hash for the given data."""
    return hashlib.sha256(str(data).encode()).hexdigest()

def initialize_vertex_hashes(graph):
    """Initialize hash values based on vertex data (or label)."""
    return {v: hash_function(graph.nodes[v]['data']) for v in graph.nodes}

def update_vertex_hashes(graph, vertex_hashes):
    """Update hash values based on the hashes of the neighboring vertices."""
    new_hashes = {}
    for v in graph.nodes:
        # Combine the hash of the vertex with the hashes of its neighbors
        neighbor_hashes = sorted([vertex_hashes[w] for w in nx.neighbors(graph, v)])
        combined_data = str(vertex_hashes[v]) + str(neighbor_hashes)
        new_hashes[v] = hash_function(combined_data)
    return new_hashes

def graph_hashing_algorithm(graph, max_iterations=100):
    """Main function to execute the graph hashing algorithm with a limit on iterations."""
    # Step 1: Initialize hash values for each vertex
    vertex_hashes = initialize_vertex_hashes(graph)
    iteration = 0
    
    while True:
        # Step 2: Update hashes
        new_vertex_hashes = update_vertex_hashes(graph, vertex_hashes)
        
        # Step 3: Check for convergence or max iterations
        if new_vertex_hashes == vertex_hashes or iteration >= max_iterations:
            break
        
        vertex_hashes = new_vertex_hashes
        iteration += 1
    
    return vertex_hashes

# Example graph creation
G = nx.Graph()
G.add_nodes_from([
    (1, {'data': 'apple'}),
    (2, {'data': 'banana'}),
    (3, {'data': 'cherry'})
])
G.add_edges_from([(1, 2), (2, 3)])

# Execute the graph hashing algorithm
final_hashes = graph_hashing_algorithm(G)
print("Final hashes of the vertices:")
for vertex, hash_value in final_hashes.items():
    print(f"Vertex {vertex}: {hash_value}")


Final hashes of the vertices:
Vertex 1: f75cf1d9da41d4814eb89b549368ef511bb42ff9624237ce4ebf71589623d87e
Vertex 2: ca5633368f7e2ae486043e24a103720e5b792b1ad1a25c2db702a34c96fbea2b
Vertex 3: d9b0f9533c91705c028a6ec9de7dfa8c3af6ef95efadd51d149e945a71a5a538


In [4]:
%matplotlib inline
import networkx as nx
import hashlib
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

def hash_function(data):
    """Generate a hash for the given data using SHA-256 and return as integer for coloring."""
    return int(hashlib.sha256(str(data).encode()).hexdigest(), 16)  # Convert hash to integer

def initialize_vertex_hashes(graph):
    """Initialize hash values for each vertex based on its label."""
    initial_hashes = {v: hash_function(v) for v in graph.nodes}
    print("Initial vertex hashes:", initial_hashes)
    return initial_hashes

def update_vertex_hashes(graph, vertex_hashes):
    """Update hash values based on the hashes of the neighboring vertices."""
    new_hashes = {}
    for v in graph.nodes:
        neighbor_hashes = sorted([vertex_hashes[w] for w in nx.neighbors(graph, v)])
        combined_data = str(vertex_hashes[v]) + ''.join(neighbor_hashes)
        new_hashes[v] = hash_function(combined_data)
    print("Updated vertex hashes:", new_hashes)
    return new_hashes

import matplotlib.colors as mcolors

def draw_graph(graph, vertex_hashes, iteration):
    """Draw the graph with vertices colored by their hash values."""
    clear_output(wait=True)  # Clear the previous plot
    plt.figure(figsize=(8, 8))
    pos = nx.spring_layout(graph, seed=42)  # Consistent positioning

    # Normalize the color scale based on hash values
    norm = mcolors.Normalize(vmin=min(vertex_hashes.values()), vmax=max(vertex_hashes.values()), clip=True)
    node_colors = [vertex_hashes[v] for v in graph.nodes()]

    # Create a colormap
    cmap = plt.cm.viridis

    # Draw the graph
    nx.draw_networkx_nodes(graph, pos, node_color=node_colors, node_size=700, cmap=cmap, norm=norm)
    nx.draw_networkx_edges(graph, pos)
    nx.draw_networkx_labels(graph, pos)
    plt.title(f"Iteration {iteration}")
    plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), ax=plt.gca(), shrink=0.7)
    plt.axis('off')
    plt.show()


def graph_hashing_visualization(graph, max_iterations=10):
    """Visualize the WL hashing process with debugging."""
    vertex_hashes = initialize_vertex_hashes(graph)
    for iteration in range(max_iterations + 1):
        print(f"Iteration {iteration}")
        draw_graph(graph, vertex_hashes, iteration)
        new_vertex_hashes = update_vertex_hashes(graph, vertex_hashes)
        if new_vertex_hashes == vertex_hashes:
            print("Hashes have stabilized; terminating early.")
            break
        vertex_hashes = new_vertex_hashes

# Example graph
G = nx.cycle_graph(6)
graph_hashing_visualization(G)


TypeError: draw_networkx_nodes() got an unexpected keyword argument 'norm'

<Figure size 800x800 with 0 Axes>